***GAME ANALYTICS***

In [1]:
#!pip install requests
#!pip install sqlalchemy
#!pip install psycopg2
import pandas as pd

***COMPETITION DATA FROM THE API ENDPOINTS***

In [ ]:
#Data from competitions

import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=a08eFD4gW6W90MmstinncNcOJsJBht3e3PLwWb8q"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)




In [ ]:
# Storing json data into another variable
competitions_json = response.text

#Converting json data into a dictionary format
comp_dict=json.loads(competitions_json)

#Normalizing nested json data into structured format 
competitions_nor=pd.json_normalize(comp_dict,['competitions'],max_level=2)

#Normalized data converted into a Dataframe
competitions_df=pd.DataFrame(competitions_nor)


In [ ]:
#Renaming a column 
competitions_df.rename(columns={'category.id':'category_id'},inplace=True)


In [ ]:
#Renaming a column 
competitions_df.rename(columns={'category.name':'category_name'},inplace=True)


In [ ]:
#Renaming a column 
competitions_df.rename(columns={'name':'competition_name'},inplace=True)


In [ ]:
#Renaming a column 
competitions_df.rename(columns={'id':'competition_id'},inplace=True)


In [ ]:
#Seprating categories table from dataframe
Categories_table =competitions_df[['category_id','category_name']]
Categories_table

In [49]:
#Grouping categories to find unique categories 
group_Categories_table=Categories_table.category_name.groupby(Categories_table.category_id).value_counts()

In [ ]:
#grouped categories converted into data frame
group_Categories_table_df=pd.DataFrame(group_Categories_table)

In [ ]:
#Removing column="count" from the group_Categories_table_df 
final_Categories_table = group_Categories_table_df.drop(columns=['count'])

In [ ]:
#Adding index column to the group_Categories_table_df 
final_Categories_table = final_Categories_table.reset_index()

In [ ]:

#importing Categories_table to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="categories_table"

final_Categories_table.to_sql(table_name,engine,if_exists="replace",index= False)

print("Data Transferred Successfully")



In [ ]:
#Seprating Competitions table from dataframe
competitions_table =competitions_df[['competition_id','competition_name','parent_id','type','gender','category_id']]

In [ ]:
#importing competitions_table to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="competitions_table"

competitions_table.to_sql(table_name,engine,if_exists="replace",index= False)

print("Data Transferred Successfully")

***COMPLEXES DATA FROM THE API ENDPOINTS***

In [ ]:
#Data from complexes

import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=a08eFD4gW6W90MmstinncNcOJsJBht3e3PLwWb8q"

headers = {"accept": "application/json"}

response1 = requests.get(url, headers=headers)

print(response1.text)


In [ ]:
import json
# Storing json data into another variable
complexes_json = response1.text
#Converting json data into a dictionary format
complexes_dict=json.loads(complexes_json)
#Normalizing nested json data into structured format 
complexes_nor=pd.json_normalize(complexes_dict,record_path=['complexes'])

In [6]:
#Seprating each column nested inside the venues
#venue_id
complexes_nor["venues"][0]
venue_id=[]
for i in complexes_nor["venues"]:
    try:
        venue_id.append(i[0]["id"])

    except:
        venue_id.append("Na")

complexes_nor["venue_id"]=venue_id

In [7]:
#venue_name

venue_name=[]
for i in complexes_nor["venues"]:
    try:
        venue_name.append(i[0]["name"])
    except:
        venue_name.append("Na")    

complexes_nor["venue_name"] = venue_name         

In [8]:
#city_name

city_name=[]

for i in complexes_nor["venues"]:
    try:
        city_name.append(i[0]["city_name"])
    except:
        city_name.append("Na")    

complexes_nor["city_name"] = city_name   

In [9]:
#country_name

country_name=[]

for i in complexes_nor["venues"]:
    try:
        country_name.append(i[0]["country_name"])
    except:
        country_name.append("Na")    

complexes_nor["country_name"] = country_name   


In [10]:
#country_code

country_code=[]

for i in complexes_nor["venues"]:
    try:
        country_code.append(i[0]["country_code"])
    except:
        country_code.append("Na")    

complexes_nor["country_code"] = country_code   

In [11]:
#timezone

timezone=[]

for i in complexes_nor["venues"]:
    try:
        timezone.append(i[0]["timezone"])
    except:
        timezone.append("Na")    

complexes_nor["timezone"] = timezone   



In [12]:
# dropping venues column after the sepration
complexes_final=complexes_nor.drop(columns=["venues"])

In [13]:
#Renaming a column 
complexes_final.rename(columns={'id':'complex_id'},inplace=True)

In [14]:
#Renaming a column 
complexes_final.rename(columns={'name':'complex_name'},inplace=True)

In [21]:
#Seprating complexes_table from dataframe
Complexes_Table=complexes_final[['complex_id','complex_name']]

In [ ]:
#importing Complexes_Table Table to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="complexes_table"

Complexes_Table.to_sql(table_name,engine,if_exists="replace",index= False)

print('Data Transferred Successfully')

In [22]:
#Seprating venues_table from dataframe
Venues_Table =complexes_final[['venue_id','venue_name','city_name','country_name','country_code','timezone','complex_id']]

In [25]:
# Droping row with null values
Venues_Table1 = Venues_Table[Venues_Table['venue_id'] != 'Na']
#Converting into a dataframe
Venues_Table1_final = pd.DataFrame(Venues_Table1)

In [ ]:
#importing Venues_Table Table to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="venues_table"

Venues_Table1_final.to_sql(table_name,engine,if_exists="replace",index= False)

print('Data Transferred Successfully')


***DOUBLES COMPETITOR RANKINGS DATA FROM THE API ENDPOINTS***

In [66]:
#Data from Doubles competitor rankings
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=a08eFD4gW6W90MmstinncNcOJsJBht3e3PLwWb8q"

headers = {"accept": "application/json"}

response2 = requests.get(url, headers=headers)

In [ ]:
# Storing json data into another variable
competitor_rankings =response2.text
#Converting json data into a dictionary format
cr_dict=json.loads(competitor_rankings)
#Normalizing nested json data into structured format 
cr_nor=pd.json_normalize(cr_dict,record_path=['rankings','competitor_rankings'])
#Normalized data converted into a Dataframe
cr_df=pd.DataFrame(cr_nor)


In [69]:
#Renaming a column
cr_df.rename(columns={'competitor.id':'competitor_id'},inplace = True)

In [70]:
#Renaming a column
cr_df.rename(columns={'competitor.name':'name'},inplace = True)

In [71]:
#Renaming a column
cr_df.rename(columns={'competitor.country':'country'},inplace = True)

In [72]:
#Renaming a column
cr_df.rename(columns={'competitor.country_code':'country_code'},inplace = True)

In [73]:
#Renaming a column
cr_df.rename(columns={'competitor.abbreviation':'abbreviation'},inplace = True)


In [74]:
#adding new column rank_id
cr_df['rank_id'] =cr_df.index + 1

In [78]:
#Seprating Competitor_Rankings table from dataframe
cr_df_rank=cr_df[['rank_id','rank','movement','points','competitions_played','competitor_id']]

In [ ]:
#importing Competitor_Rankings Table to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="competitor_rankings"

cr_df_rank.to_sql(table_name,engine,if_exists="replace",index= False)
print('Data Transferred Successfully')


In [ ]:
#Seprating competitors_table from dataframe
competitors = cr_df[['competitor_id','name','country','country_code','abbreviation']]

In [ ]:
#importing Competitors Table  to sports database
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(conn_string)

table_name ="competitors_table"

competitors.to_sql(table_name,engine,if_exists="replace",index= False)
print('Data Transferred Successfully')

***Inserting complete data***

In [ ]:
#Tennis_Tournaments

from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine= create_engine(conn_string)

table_name='tennis_tournaments'

competitions_df.to_sql(table_name,engine,if_exists="replace",index=False)
print('Data Transferred Successfully')

In [ ]:
#Points Table

from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine= create_engine(conn_string)

table_name='points_details'

cr_df.to_sql(table_name,engine,if_exists="replace",index=False)
print('Data Transferred Successfully')

In [ ]:
#tennis_courts

from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "game_analytics"

conn_string =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine= create_engine(conn_string)

table_name='tennis_courts'

complexes_final.to_sql(table_name,engine,if_exists="replace",index=False)
print('Data Transferred Successfully')